In [147]:
import pandas as pd
import numpy as np

## Iris dataset
The Iris dataset is comprised of 150 instances of Iris flowers from three different species  (classes),  where  each  class  refers  to  a  type  of Iris  plant.  There  are  4 attributes of given flowers: sepal length, sepal width, petal length and petal width, 
all in the same unit of centimeters.

The predicted attribute is the species, which is one of Setosa, Versicolor and Virginica. 

In [148]:
df = pd.read_csv("IRIS-2.csv",header=None)
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Load data
Shuffle and split the IRIS-dataset into train/test datasets (with 66%/34% split).
* X_train: contains  all  features  from the training set
* y_train: contains the class label (flower species) of the training set
* X_test: contains all features of the testset
* y_test: contains the class label of the testset

In [149]:
def load_data(path):
    df = pd.read_csv(path,header=None)
    df = df.sample(frac=1) #Shuffles the dataset
    X_train = df.iloc[:100, :4] #66%
    X_test = df.iloc[100:150,:4] #34%
    y_train = df.iloc[:100, 4]
    y_test = df.iloc[100:150,4]
    return X_train, X_test, y_train, y_test

### Euclidean distance
$$Euclidean\, Distance \, (x,y) = \sqrt{\sum_{i=1}^n (x_{i}-y_{i})^2}$$

Calculate the euclidean distance between a test object (x) and all instances in the training dataset.
np.linalg.norm with axis=1 performs vector norm calculations between the testobject and trainingset. The function then returns the matrix norm that contains every euclidean distance betweem them.

In [150]:
def euclidean_distance(testobject,trainingset):
    return np.linalg.norm(testobject - trainingset, axis=1)

### Find neighbors
Locate the k most similar data instances to the test object, based on the computed euclidean distance vector. 
The distance vector is sorted in ascendic order, and then sliced into the k-first elements.

In [151]:
def find_neighbors(k,array):
    return np.argsort(array)[:k] #Returns the indices that would sort an array    

### Predict the class
Assign the most common label of the k nearest neighbors to the class label of the test example. K indexes is used to look up the labels in ytrain, count them by np.bincount, before returning the most frequent one.


In [152]:
def majority_class(indexes, array):
    #count = np.bincount([array[x] for x in indexes]), then argmax(count)
    return np.argmax(np.bincount([array[x] for x in indexes])) #as a one-liner

### K-NN
Produce predictions for all instances in the test dataset. The dataframes is first converted into numpy arrays in order to performe efficient calculations.

In [162]:
#Returns a list of predictions
def k_nearest_neighbors(xtrain,xtest,ytrain,ytest,k):
    xtrain, xtest, ytrain, ytest = np.array(xtrain), np.array(xtest), np.array(ytrain), np.array(ytest)
    predictions = []
    for element in xtest:
        euclidean_distances = euclidean_distance(element,xtrain) #list of euclidean distances from the element
        neighbors = find_neighbors(k,euclidean_distances) #k nearest elements
        prediction = majority_class(neighbors,ytrain) #class of the most common label
        predictions.append(prediction)
    return predictions

In [181]:
def opt_k_nearest_neighbors(xtrain,xtest,ytrain,k):
    xtrain, xtest, ytrain = np.array(xtrain), np.array(xtest), np.array(ytrain)
    #perform every needed function in one comprehension
    return [majority_class(x,ytrain) for x in [find_neighbors(k,x) for x in [euclidean_distance(x,xtrain) for x in xtest]]]

### Accuracy
Classification accuracy is a ratio of the total correct predictions out of all predictions made. The function is simply counting each correct prediction by comparing it to the y_test (true labels). Accuracy is computed by #correct/length-testset. 

In [155]:
def model_accuracy(pred, true):
    correct = 0
    for i, prediction in enumerate(pred):
        if prediction == true[i]:
            correct += 1
    return correct/len(pred)

### Main method
Call each seperate function with an appropriate order to produce the final results

In [193]:
def main():
    xtrain, xtest, ytrain, ytest = load_data("IRIS-2.csv") #Load the IRIS dataset
    #predictions = k_nearest_neighbors(xtrain,xtest,ytrain,ytest,5) #List of predictions with k=5
    predictions = opt_k_nearest_neighbors(xtrain,xtest,ytrain,5)
    accuracy = model_accuracy(predictions,np.array(ytest))
    #print("Model accuracy: ", accuracy)

## Profiling

### Main method
The main method loads, predict and computes the accuracy of the KNN algorithm. I used the built in magic command %timeit to profile my code. The %timeit command measures more accurate results than the regular %time as it repeats execution of a single statement. My results from the main method:
* 5.27 ms ± 301 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) (with print of accuracy)
* 4.85 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) (without print of accuray)
* 4.08 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) (with the one-liner method opt_k_nearest_neighbors)

This shows how efficient numpy calculations are. Removing the print statement also reduced the run time. My alternative KNN method opt_k_nearest_neighbors() runs faster than the regular one. The method is however not as clear and understandable as the other one.

To go more in depth of where the algorithm spend most of it execution time, I look at different parts by itself. Pandas is used to load the dataset into dataframes in load_data(). This part is pretty straightforward and there is not much more to do in terms of efficiency. Looking at the execution time of the load_data() I get:
* 3.11 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

The entire method then spend most of its time loading the data. This means all the calculations after the data is loaded, runs in under 1.0 ms. Checking it out with %timeit, I get:
* 1.07 ms ± 7.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each) (For calculating predictions)
* 34 µs ± 773 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each) (For calculating the accuracy)

The KNN calculations is completed in 1.07ms. This is the measurements of the key parts within the function (one xtest element):
* 8.77 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) (For calculation the euclidean distances)
* 2.64 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) (For calculating the nearest neighbors)
* 4.7 µs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) (For finding majority class)
* 4.62 µs ± 94.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) (For append prediction to list (regular knn methos)


In [195]:
%timeit main()

4.09 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
